In [ ]:
import math
import sys

import IPython
import IPython.display as ipd
import matplotlib.pylab as plt
import numpy as np
import pandas as pd

%reload_ext autoreload
%autoreload 2

%matplotlib inline
#%matplotlib notebook

from matplotlib import rcParams
rcParams["figure.max_open_warning"] = False

In [ ]:
from wall_analysis import parse_experiments
#exp_name = '2020_12_18_stepper'; appendix = ""; distance = 51
#exp_name = '2020_12_18_flying'; appendix="_new"; distance = 0
#exp_name = '2021_03_01_flying';
exp_name = '2021_04_30_hover';
fname = f'../experiments/{exp_name}/all_data.pkl'

try:
    df_total = pd.read_pickle(fname)
    print('read', fname)
except:
    answer = input('Run wall_analysis.py to parse experiments? (y/[n])') or 'n'
    if answer == 'y':
        df_total = parse_experiments(exp_name)
        pd.to_pickle(df_total, fname)
        print('saved', fname)

In [ ]:
df_total

In [ ]:
import seaborn as sns
from plotting_tools import save_fig

min_time = 0
max_time = 7.5

max_dist = 50 # cm
for i, row in df_total.iterrows():
    print('starting position:', row.positions[0, :])
    positions_cm = row.positions[:, :3] * 100
    
    fig, axs = plt.subplots(1, 3) 
    fig.set_size_inches(10, 3.3)
    fig.suptitle(row.appendix, y=1.0)
    
    mask_time = (row.seconds > min_time) & (row.seconds < max_time)
    time = row.seconds[mask_time]
    
    #axs[0].plot(x=positions_cm[:, 0], y=positions_cm[:, 1], color=colors())
    sns.scatterplot(x=positions_cm[mask_time, 0], y=positions_cm[mask_time, 1], 
                    hue=time, ax=axs[0], linewidth=0, 
                    #size=positions_cm[:, 2],
                    palette='inferno')
    axs[0].set_xlabel('x [cm]')
    axs[0].set_ylabel('y [cm]')
    #axs[0].set_xlim(-max_dist/2, max_dist/2)
    #axs[0].set_ylim(-max_dist/2, max_dist/2)
    axs[0].axis('equal')
    axs[0].legend(loc='lower right', title='time [s]')

    axs[1].plot(time, positions_cm[mask_time, 0], label='x')
    axs[1].plot(time, positions_cm[mask_time, 1], label='y')
    axs[1].plot(time, positions_cm[mask_time, 2], label='z')
    axs[1].set_xlabel('time [s]')
    axs[1].set_ylabel('movement [cm]')
    axs[1].set_ylim(-max_dist, max_dist)
    axs[1].legend(loc='lower right')

    axs[2].plot(time, row.positions[mask_time, 3], label='yaw')
    axs[2].set_ylabel('yaw [deg]')
    axs[2].set_xlabel('time [s]')
    axs[2].set_ylim(-20, 20)
    axs[2].legend(loc='lower right')

    plt.tight_layout()
    #save_fig(fig, f'plots/{exp_name}{row.appendix}_movement.pdf')

# Analysis

In [ ]:
from frequency_analysis import add_spectrogram

df_total = df_total.assign(spectrogram=None)
df_total = df_total.apply(add_spectrogram, axis=1)

print(df_total.appendix.unique())
print(df_total.source.unique())
print(df_total.distance.unique())

In [ ]:
mic_idx = 0
min_freq = 2900 
max_freq = 5000

#maxi = np.nanmax(np.concatenate([*dfs.spectrogram], axis=1))
for i_col, row in df_total.iterrows():
    all_frequencies = np.unique(row.frequencies_matrix)
    spec = row.spectrogram[:, mic_idx, :]
    print(spec.shape, spec.dtype)
    spec[spec == 0] = np.nan
    total = np.nanmean(np.abs(spec), axis=1)
    
    label = str(f"{row.appendix}").replace('_', '')
    fig, ax = plt.subplots()
    ax.set_title(f'spectrogram of mic{mic_idx}, appendix {row.appendix}')
    fig.set_size_inches(5, 5)
    ax.pcolorfast(row.seconds, 
                  all_frequencies, np.abs(spec)[:-1, :-1])
    ax.set_ylabel('frequency [Hz]')
    ax.set_ylim(min_freq, max_freq)
    ax.set_xlim(min_time, max_time)

In [ ]:
#spec_masked, freqs_masked = wall_detector.fill_from_row()
from wall_detector import WallDetector
row = df_total.iloc[0]
wall_detector = WallDetector(exp_name=exp_name)

print(row.stft.shape)
print(row.frequencies_matrix.shape)

signals_f_all = row.stft
frequencies_all = row.frequencies_matrix

fig, axs = plt.subplots(1, signals_f_all.shape[1])
fig.set_size_inches(15, 5)
for i in range(signals_f_all.shape[0]):
    signals_f = signals_f_all[i]
    frequencies = frequencies_all[i]
    
    for i_mic in range(signals_f.shape[0]):
        axs[i_mic].plot(frequencies, np.abs(signals_f[i_mic, :]),
                        label=f'mic{i_mic}')
        
    wall_detector.fill_from_signal(signals_f, frequencies)
wall_detector.to_numeric()

In [ ]:
wall_detector.cleanup_conservative(verbose=True)

In [ ]:
import seaborn as sns
sns.set_palette('tab10')
sns.scatterplot(
    data=wall_detector.df, x='frequency', y='magnitude', hue='mic',
    linewidth=0.0
)
wall_detector.df.loc[(wall_detector.df.mic==i_mic)
                   & (wall_detector.df.distance==0)
                   & (wall_detector.df.angle==0)
                   & (wall_detector.df.frequency==3515)]
#n_freqs = wall_detector.remove_bad_freqs(verbose=True)
print(n_freqs)

In [ ]:
from dataset_parameters import kwargs_datasets
from simulation import get_df_theory


kwargs = kwargs_datasets[exp_name]["audio_deck"]
distances_grid = np.arange(30, 80)
frequencies = np.linspace(kwargs["min_freq"], kwargs["max_freq"], 30)

df_matrix_theo = get_df_theory(frequencies, distances_grid, mic_idx)

In [ ]:
from wall_detector import WallDetector, get_probability_cost, get_probability_fft
from calibration import get_calibration_function_dict
from frequency_analysis import get_index_matrix
from plotting_tools import add_colorbar

distance_dict = {
    '_30cm-paper': 42,
    '_30cm-nopaper': 42,
    '_30cm-newbuzzer': 42,
    '_50cm-paper': 57,
    '_50cm-nopaper': 65,
    '_50cm-newbuzzer': 70,
}

appendix_plot = '-newbuzzer'
#appendix_plot = '-paper'
#appendix_plot = '-nopaper'
for appendix_plot in ['-newbuzzer']:#,'-paper','-nopaper']:
    
    for _, row in df_total.iterrows():
        if not appendix_plot in row.appendix:
            continue

        fig, ax = plt.subplots()
        if 'newbuzzer' in row.appendix: 
            calib_function = get_calibration_function_dict(exp_name="2021_02_25_wall", mic_type="audio_deck", motors=0, ax=ax)
        else:
            calib_function = get_calibration_function_dict(exp_name="2021_02_23_wall", mic_type="audio_deck", motors=0, ax=ax)
            
        nominal_distance = distance_dict[row.appendix]
        label = str(f"{row.appendix}").replace('_', '')
        
        wall_detector = WallDetector(exp_name=exp_name)
        spec_masked, freqs_masked = wall_detector.fill_from_row(row)
        
        ### extract times of interest ###
        time_mask = (row.seconds > kwargs["min_time"]) & (row.seconds < kwargs["max_time"]) 
        spec_masked = spec_masked[:, :, time_mask]
        seconds = row.seconds[time_mask]

        ### extract varying slice ### 
        index_matrix = get_index_matrix(spec_masked)
        strongest_freqs = freqs_masked[index_matrix[0, :]]
        f_slice_varying = [spec_masked[f_idx, mic_idx, idx] for idx, f_idx in enumerate(index_matrix[0, :])]
        relative_ds = - row.positions[time_mask, 1] * 100
        absolute_yaws = row.positions[time_mask, 3]
        distances_y = nominal_distance + relative_ds

        wall_detector.remove_bad_freqs(verbose=True)
        wall_detector.remove_spurious_freqs(verbose=True)
        f_slice, freqs = wall_detector.get_frequency_slice()
        f_slice = f_slice[mic_idx]
        f_calib = calib_function(freqs)[mic_idx]
        
        fig, ax_slices = plt.subplots()
        fig.set_size_inches(5, 5)
        ax_slices.plot(freqs, f_slice, label=label)
        ax_slices.plot(freqs, f_calib, label=label + ' calib')
        ax_slices.scatter(strongest_freqs, f_slice_varying)

        ### calibrate ###
        f_slice_varying_norm = f_slice_varying / calib_function(strongest_freqs)[mic_idx]
        f_slice_norm = f_slice / calib_function(freqs)[mic_idx]
        f_slice_varying_theory = get_freq_slice_theory(strongest_freqs, distances_y)[:, mic_idx]

        fig, axs = plt.subplots(2, 1)
        fig.set_size_inches(10, 5)
        f_slice_norm -= np.mean(f_slice_norm)
        f_slice_norm /= np.std(f_slice_norm)
        f_slice_varying -= np.mean(f_slice_varying)
        f_slice_varying /= np.std(f_slice_varying)
        f_slice_varying_theory -= np.mean(f_slice_theory)
        f_slice_varying_theory /= np.std(f_slice_theory)
        f_slice_varying_norm -= np.mean(f_slice_varying_norm)
        f_slice_varying_norm /= np.std(f_slice_varying_norm)

        axs[0].plot(seconds, f_slice_varying, label='experimental')
        axs[0].plot(seconds, f_slice_varying_norm, label='experimental calibrated')
        axs[0].plot(seconds, f_slice_varying_theory, label='theoretical')
        axs[0].legend()
        axs[0].grid(which='both')
        axs[1].plot(seconds, strongest_freqs)
        axs[1].grid(which='both')
    
    
        ##### algorithm #####
        #proba_cost = get_probability_cost(
        #    f_slice, freqs, distances_grid, mic_idx=mic_idx
        #)
        #proba_cost_norm = get_probability_cost(
        #    f_slice_norm, freqs, distances_grid, mic_idx=mic_idx
        #)
        proba_cost_mvmt = get_probability_cost(
            f_slice_varying, strongest_freqs, distances_grid, mic_idx=mic_idx, relative_ds=relative_ds, absolute_yaws=absolute_yaws
        )
        proba_cost_mvmt_norm = get_probability_cost(
            f_slice_varying_norm, strongest_freqs, distances_grid, mic_idx=mic_idx, relative_ds=relative_ds, absolute_yaws=absolute_yaws
        )
        proba_cost_mvmt_theory = get_probability_cost(
            f_slice_varying_theory, strongest_freqs, distances_grid, mic_idx=mic_idx, relative_ds=relative_ds, absolute_yaws=absolute_yaws
        )
        fig_cost, ax_proba_cost = plt.subplots()
        fig.set_size_inches(5, 5)
        #ax_proba_cost.plot(distances_grid, proba_cost, label=label)
        #ax_proba_cost.plot(distances_grid, proba_cost_norm, label=label + ' calibrated')
        ax_proba_cost.plot(distances_grid, proba_cost_mvmt, label=label)
        ax_proba_cost.plot(distances_grid, proba_cost_mvmt_norm, label=label + ' calibrated')
        ax_proba_cost.plot(distances_grid, proba_cost_mvmt_theory, label=label + ' theory')
        ax_proba_cost.axvline(nominal_distance, label=f'ground truth {label}', color='black')
        ax_proba_cost.legend(loc='upper left', bbox_to_anchor=[1.0, 1.0]) 
        ax_proba_cost.set_ylabel('probability')
        ax_proba_cost.set_xlabel('distance [cm]')
        ax_proba_cost.set_yscale('log')
        
        ### full df ###
        fig, ax = plt.subplots()
        ax.pcolorfast(distances_grid, frequencies, df_matrix_theo[:-1, :-1])
        ax.scatter(distances_y, strongest_freqs, color='C1', s=5)
        ax.axvline(nominal_distance, color='C1')
        ax.set_ylim(min(frequencies), max(frequencies))
        ax.set_title(f'DF of mic{mic_idx}, {label}')
        ax.set_xlabel('distance [cm]')
        ax.set_ylabel('frequency [Hz]')
        save_fig(fig, f'plots/{exp_name}{row.appendix}_df.pdf')
        
        ### small df ###
        #d_sorted = np.sort(distances_y)
        #f_sorted = np.sort(strongest_freqs)
        d_sorted = np.arange(min(distances_y), max(distances_y), step=0.1) #np.sort(distances_y)
        f_sorted = np.arange(min(strongest_freqs), max(strongest_freqs), step=50) #np.sort(strongest_freqs)
        df_small = get_df_theory(f_sorted, d_sorted, mic_idx)
        fig, axs = plt.subplots(1, 2, sharey=True)
        axs[0].pcolorfast(d_sorted, f_sorted, df_small[:-1, :-1])
        
        frequency_indices = np.argmin(np.abs(f_sorted[:, None] - strongest_freqs[None, :]), axis=0)
        #np.testing.assert_allclose(f_sorted[frequency_indices], strongest_freqs)
        distance_indices = np.argmin(np.abs(d_sorted[:, None] - distances_y[None, :]), axis=0)
        df_small[frequency_indices, distance_indices] = f_slice_varying
        axs[1].pcolorfast(d_sorted, f_sorted, df_small[:-1, :-1])

    #ax_proba_fft.set_xlim(min(distances_grid), max(distances_grid))
    #save_fig(fig_cost, f'plots/{exp_name}{appendix_plot}_cost.pdf')
    ax_slices.legend(loc='upper left')